In [8]:
# # `REINVENT 4.4`: Reinforcement Learning with DockStream and (docking)

In [9]:
import os
import json
import toml

In [10]:
dockstream_path = os.path.expanduser("/Users/devanshjain/DockStream")
dockstream_env = os.path.expanduser("/Users/devanshjain/miniconda3/envs/DockStream")

In [11]:
apo_protein_filename = "/Users/devanshjain/7xn1_apo.pdb"  # Change this to your apo protein file name
reference_ligand_filename = "/Users/devanshjain/7xn1_tacrine.pdb"  # Change this to your reference ligand file name

In [12]:
output_prefix = "p7xn1"
project_dir = os.path.expanduser("/Users/devanshjain/laboratoire_d_intelligence_artificielle_en_chimie")

In [13]:
target_preparator = os.path.join(dockstream_path, "target_preparator.py")
docker = os.path.join(dockstream_path, "docker.py")

input_data_dir = os.path.join(project_dir, "input_data")
output_data_dir = os.path.join(project_dir, "output_data")
logs_dir = os.path.join(project_dir, "logs")
config_dir = os.path.join(project_dir, "configs")
lig_docked_dir = os.path.join(output_data_dir, "ligands_docked")
scores_dir = os.path.join(output_data_dir, "docking_scores")

# Create necessary directories
for directory in [input_data_dir, output_data_dir, logs_dir, config_dir, lig_docked_dir, scores_dir]:
    os.makedirs(directory, exist_ok=True)

# Update file paths
apo_protein_path = os.path.join(input_data_dir, apo_protein_filename)
reference_ligand_path = os.path.join(input_data_dir, reference_ligand_filename)

target_prep_path = os.path.join(config_dir, f"{output_prefix}_target_prep.json")
fixed_pdb_path = os.path.join(input_data_dir, f"{output_prefix}_fixed_target.pdb")
receptor_path = os.path.join(input_data_dir, f"{output_prefix}_receptor.pdbqt")
#receptor_path_oedu = os.path.join(input_data_dir, f"{output_prefix}_receptor.oedu")
log_file_target_prep = os.path.join(logs_dir, f"{output_prefix}_target_prep.log")
log_file_docking = os.path.join(logs_dir, f"{output_prefix}_docking.log")
log_file_reinvent = os.path.join(logs_dir, f"{output_prefix}_reinvent.log")

docking_path = os.path.join(config_dir, f"{output_prefix}_docking.json")
ligands_docked_path = os.path.join(lig_docked_dir, f"{output_prefix}_ligands_docked.sdf")
ligands_scores_path = os.path.join(scores_dir, f"{output_prefix}_scores.csv")
ligands_conformer_path = os.path.join(lig_docked_dir, f"{output_prefix}pydantic.sdf")

In [62]:
tp_dict = {
  "target_preparation":
  {
    "header": {                                   # general settings
      "environment": {
      },
      "logging": {                                # logging settings (e.g. which file to write to)
        "logfile": log_file_target_prep
      }
    },
    "input_path": apo_protein_path,                  # this should be an absolute path
    "fixer": {                                    # based on "PDBFixer"; tries to fix common problems with PDB files
      "enabled": True,
      "standardize": True,                        # enables standardization of residues
      "remove_heterogens": True,                  # remove hetero-entries
      "fix_missing_heavy_atoms": True,            # if possible, fix missing heavy atoms
      "fix_missing_hydrogens": True,              # add hydrogens, which are usually not present in PDB files
      "fix_missing_loops": False,                 # add missing loops; CAUTION: the result is usually not sufficient
      "add_water_box": False,                     # if you want to put the receptor into a box of water molecules
      "fixed_pdb_path": fixed_pdb_path            # if specified and not "None", the fixed PDB file will be stored here
    },
    "runs": [                                     # "runs" holds a list of backend runs; at least one is required
      {
        "backend": "AutoDockVina",                # one of the backends supported ("AutoDockVina", "OpenEye", ...)
        "output": {
          "receptor_path": receptor_path      # the generated receptor file will be saved to this location
        },
        "parameters": {
            "extract_box": {                        # in order to extract the coordinates of the pocket (see text)
            "reference_ligand_path": reference_ligand_path,   # path to the reference ligand
            "reference_ligand_format": "pdb"                  # format of the reference ligand
          }},
        "cavity": {                               # there are different ways to specify the cavity; here, a reference
                                                  # ligand is used
          "method": "reference_ligand",
          "reference_ligand_path": reference_ligand_path,
          "reference_ligand_format": "pdb"
}}]}}

with open(target_prep_path, 'w') as f:
    json.dump(tp_dict, f, indent=2)

In [63]:
!{dockstream_env}/bin/python {target_preparator} -conf {target_prep_path}

In [64]:
!head -n 25 {receptor_path}

REMARK  Name = /var/folders/jv/219p6n9s5ll8vv2nm1whxw340000gn/T/tmpwyvcyx6i.pdb
REMARK                            x       y       z     vdW  Elec       q    Type
REMARK                         _______ _______ _______ _____ _____    ______ ____
ATOM      1  N   PRO A   1      74.496 -47.808   0.857  0.00  0.00    +0.386 N 
ATOM      2  CA  PRO A   1      74.139 -46.789  -0.149  0.00  0.00    -0.005 C 
ATOM      3  C   PRO A   1      75.079 -46.679  -1.396  0.00  0.00    +0.199 C 
ATOM      4  O   PRO A   1      76.289 -46.874  -1.306  0.00  0.00    -0.278 OA
ATOM      5  CB  PRO A   1      74.074 -45.415   0.542  0.00  0.00    -0.048 C 
ATOM      6  CG  PRO A   1      73.196 -44.401  -0.185  0.00  0.00    +0.048 C 
ATOM      7  CD  PRO A   1      71.754 -44.309   0.370  0.00  0.00    +0.356 C 
ATOM      8  OE1 PRO A   1      71.565 -43.889   1.526  0.00  0.00    -0.246 OA
ATOM      9  OE2 PRO A   1      70.797 -44.631  -0.366  0.00  0.00    -0.246 OA
ATOM     10  N   GLN A   2      74.5

In [14]:
vina_binary_location = os.path.expanduser("/Users/devanshjain/Downloads/autodock_vina_1_1_2_mac_catalina_64bit/bin")
smiles_path = os.path.expanduser("/Users/devanshjain/smiles.smi")

In [66]:
with open(smiles_path, 'r') as f:
    smiles = [smile.strip() for smile in f.readlines()]
print(smiles)

['C1=CC=C2C(=C1)C(=NC=C2)N', 'CCOC(=O)C1=CC=CC=C1CN1CCN(CC1)C', 'CCOC(=O)C1CCN(C)CC1', 'CN1CCC23C4CC5=CC=CC=C5C4CC2C1C3', 'CC1=CC2C3CCN4CCC(C3C2N1)C4']


In [15]:
ed_dict = {
  "docking": {
    "header": {
      "environment": {},
      "logging": {"logfile": log_file_docking}
    },
    "ligand_preparation": {
      "embedding_pools": [
        {
          "pool_id": "RDkit",
          "type": "RDkit",
          "parameters": {
            "protonate": True,
            "remove_hs": False,
            "coordinate_generation": {
              "method": "UFF",
              "maximum_iterations": 600
            }
          },
          "input": {
            "input_path": smiles_path,
            "type": "smi",
            "standardize_smiles": False
          },
          "use_taut_enum": {
            "prefix_execution": "module load taut_enum",
            "enumerate_protonation_states": True
          },
          "output": {                                   # the conformers can be written to a file, but "output" is
                                                        # not required as the ligands are forwarded internally
            "conformer_path": ligands_conformer_path, 
            "format": "sdf"
          }
        }
      ]
    },
    "docking_runs": [
      {
        "backend": "AutoDockVina",
        "run_id": "AutoDockVina",
        "input_pools": ["RDkit"],
        "parameters": {
          "binary_location": vina_binary_location,
          "parallelization": {"number_cores": 4},
          "seed": 42,
          "receptor_pdbqt_path": [receptor_path],
          "number_poses": 1,
          "search_space": {
              "--center_x": 53.637,   # Example for 7XN1
              "--center_y": -29.391,
              "--center_z": -8.917,
              "--size_x": 36,
              "--size_y": 25,
              "--size_z": 14
            }
        },
        "output": {
          "poses": {
            "poses_path": ligands_docked_path,
            "format": "sdf"
          },
          "scores": {
            "scores_path": ligands_scores_path,
            "format": "csv"
          }
        }
      }
    ]
  }
}

with open(docking_path, 'w') as f:
    json.dump(ed_dict, f, indent=2)

# print out path to generated JSON
print(docking_path)

/Users/devanshjain/laboratoire_d_intelligence_artificielle_en_chimie/configs/p7xn1_docking.json


In [76]:
!{dockstream_env}/bin/python {docker} -conf {docking_path} -print_scores

/Users/devanshjain/miniconda3/envs/DockStream/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
/Users/devanshjain/miniconda3/envs/DockStream/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'validate_by_name'
  warnings.warn(message, UserWarning)
/Users/devanshjain/miniconda3/envs/DockStream/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
[19:22:13] Molecule does not have explicit Hs. Consider calling AddHs()
[19:22:13] Molecule does not have explicit Hs. Consider calling AddHs()
[19:22:13] Molecule does not have explicit Hs. Consider calling AddHs()
[19:22:13] Mo

In [74]:
#Setting up for Reinvent

In [16]:
ed_dict = {
  "docking": {
    "header": {
      "environment": {},
      "logging": {"logfile": log_file_docking}
    },
    "ligand_preparation": {
      "embedding_pools": [
        {
          "pool_id": "RDkit",
          "type": "RDkit",
          "parameters": {
            "protonate": True,
            "remove_hs": False,
            "coordinate_generation": {
              "method": "UFF",
              "maximum_iterations": 600
            }
          },
          "input": {
            "input_path": "",
            "type": "console",
            "standardize_smiles": False
          },
          "use_taut_enum": {
            "prefix_execution": "module load taut_enum",
            "enumerate_protonation_states": True
          },
          "output": {                                   # the conformers can be written to a file, but "output" is
                                                        # not required as the ligands are forwarded internally
            "conformer_path": ligands_conformer_path, 
            "format": "sdf"
          }
        }
      ]
    },
    "docking_runs": [
      {
        "backend": "AutoDockVina",
        "run_id": "AutoDockVina",
        "input_pools": ["RDkit"],
        "parameters": {
          "binary_location": vina_binary_location,
          "parallelization": {"number_cores": 4},
          "seed": 42,
          "receptor_pdbqt_path": [receptor_path],
          "number_poses": 1,
          "search_space": {
              "--center_x": 53.637,   # Example for 7XN1
              "--center_y": -29.391,
              "--center_z": -8.917,
              "--size_x": 36,
              "--size_y": 25,
              "--size_z": 14
            }
        },
        "output": {
          "poses": {
            "poses_path": ligands_docked_path,
            "format": "sdf"
          },
          "scores": {
            "scores_path": ligands_scores_path,
            "format": "csv"
          }
        }
      }
    ]
  }
}

with open(docking_path, 'w') as f:
    json.dump(ed_dict, f, indent=2)

# print out path to generated JSON
print(docking_path)

/Users/devanshjain/laboratoire_d_intelligence_artificielle_en_chimie/configs/p7xn1_docking.json


In [ ]:
#staged learning config

In [ ]:
toml_string = f"""

{
    "run_type": "staged_learning",
    "use_cuda": true,
    "tb_logdir": "tb_logs",
    "json_out_config": "_staged_learning.json",
    "parameters": {
        "use_checkpoint": false,
        "summary_csv_prefix": "staged_learning",
        "prior_file": "/Users/devanshjain/REINVENT4/priors/reinvent.prior",
        "agent_file": "/Users/devanshjain/REINVENT4/priors/reinvent.prior",
        "batch_size": 64,
        "unique_sequences": true,
        "randomize_smiles": true
    },
    "learning_strategy": {
        "type": "dap",
        "sigma": 128,
        "rate": 0.0001
    },
    "diversity_filter": {
        "type": "IdenticalMurckoScaffold",
        "bucket_size": 25,
        "minscore": 0.4,
        "minsimilarity": 0.4,
        "penalty_multiplier": 0.5
    },
    "stage": [
        {
            "chkpt_file": "test1.chkpt",
            "termination": "simple",
            "max_score": 0.6,
            "min_steps": 25,
            "max_steps": 100,
            "scoring": {
                "type": "geometric_mean",
                "component": [
                    {
                        "custom_alerts": {
                            "endpoint": [
                                {
                                    "name": "Unwanted SMARTS",
                                    "weight": 0.79,
                                    "params": {
                                        "smarts": [
                                            "[*;r8]",
                                            "[*;r9]",
                                            "[*;r10]",
                                            "[*;r11]",
                                            "[*;r12]",
                                            "[*;r13]",
                                            "[*;r14]",
                                            "[*;r15]",
                                            "[*;r16]",
                                            "[*;r17]",
                                            "[#8][#8]",
                                            "[#6;+]",
                                            "[#16][#16]",
                                            "[#7;!n][S;!$(S(=O)=O)]",
                                            "[#7;!n][#7;!n]",
                                            "C#C",
                                            "C(=[O,S])[O,S]",
                                            "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
                                            "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
                                            "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
                                            "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
                                            "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
                                            "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
                                        ]
                                    }
                                }
                            ]
                        }
                    },
                    {
                        "MolecularWeight": {
                            "endpoint": [
                                {
                                    "name": "Molecular weight",
                                    "weight": 0.342,
                                    "transform": {
                                        "type": "double_sigmoid",
                                        "high": 500.0,
                                        "low": 200.0,
                                        "coef_div": 500.0,
                                        "coef_si": 20.0,
                                        "coef_se": 20.0
                                    }
                                }
                            ]
                        }
                    }
                ]
            }
        },
        {
            "chkpt_file": "test2.chkpt",
            "termination": "simple",
            "max_score": 0.7,
            "min_steps": 10,
            "max_steps": 100,
            "scoring": {
                "type": "geometric_mean",
                "filename": "stage2_scoring.toml",
                "filetype": "toml"
            }
        }
    ]
}

"""

In [78]:
toml_string = f"""

# REINVENT4 TOML input example for reinforcement/curriculum learning
#
#
# Curriculum learning in REINVENT4 is a multi-stage reinforcement learning
# run.  One or more stages (auto CL) can be defined.  But it is also
# possible to continue a run from any checkpoint file that is generated
# during the run (manual CL).  Currently checkpoints are written at the end
# of a run also when the run is forcefully terminated with Ctrl-C.


run_type = "staged_learning"
device = "cuda:0"  # set torch device e.g. "cpu"
tb_logdir = "tb_logs"  # name of the TensorBoard logging directory
json_out_config = "_staged_learning.json"  # write this TOML to JSON

[parameters]

summary_csv_prefix = "staged_learning"  # prefix for the CSV file
use_checkpoint = false  # if true read diversity filter from agent_file
purge_memories = false  # if true purge all diversity filter memories after each stage


## Reinvent
prior_file = "/Users/devanshjain/REINVENT4/priors/reinvent.prior"
agent_file = "/Users/devanshjain/REINVENT4/priors/reinvent.prior"

##ISIM
tb_isim = false # if true track isim value of smilies across training epochs in tensorboard 

batch_size = 64          # network

unique_sequences = true  # if true remove all duplicates raw sequences in each step
                         # only here for backward compatibility
randomize_smiles = true  # if true shuffle atoms in SMILES randomly


[learning_strategy]

type = "dap"      # dap: only one supported
sigma = 128       # sigma of the RL reward function
rate = 0.0001     # for torch.optim


[diversity_filter]  # optional, comment section out or remove if unneeded
                    # NOTE: also memorizes all seen SMILES

type = "IdenticalMurckoScaffold" # IdenticalTopologicalScaffold,
                                 # ScaffoldSimilarity, PenalizeSameSmiles
bucket_size = 25                 # memory size in number of compounds
minscore = 0.4                   # only memorize if this threshold is exceeded
minsimilarity = 0.4              # minimum similarity for ScaffoldSimilarity
penalty_multiplier = 0.5         # penalty factor for PenalizeSameSmiles


# Reinvent only: guide RL in the initial phase
#[inception]  # optional, comment sectionout or remove if unneeded

#smiles_file = "sampled.smi"  # "good" SMILES for guidance
#memory_size = 100  # number of total SMILES held in memory
#sample_size = 10  # number of SMILES randomly chosen each epoch


### Stage 1
### Note that stages must always be a list i.e. double brackets
[[stage]]

chkpt_file = 'test1.chkpt'  # name of the checkpoint file, can be reused as agent

termination = "simple"  # termination criterion fot this stage
max_score = 0.6  # terminate if this total score is exceeded
min_steps = 25  # run for at least this number of steps
max_steps = 100  # terminate entire run when exceeded

# Optionally, a DF can be set for each stage but note that the global DF
# section above will always overwrite the stage section and you need to
# delete [diversity_filter] to avoid this
#
#[stage.diversity_filter]
#type = "IdenticalMurckoScaffold"
# etc.

[stage.scoring]
type = "geometric_mean"  # aggregation function

[[stage.scoring.component]]
# Custom alerts if used in a custom_product filter out unwanted groups
[stage.scoring.component.custom_alerts]

[[stage.scoring.component.custom_alerts.endpoint]]
name = "Unwanted SMARTS"  # user chosen name for output
weight = 0.79  # weight to fine-tune the relevance of this component

# parameters for the component:
# a list of unwanted SMARTS(!) to be scored as zero
params.smarts = [
    "[*;r8]",
    "[*;r9]",
    "[*;r10]",
    "[*;r11]",
    "[*;r12]",
    "[*;r13]",
    "[*;r14]",
    "[*;r15]",
    "[*;r16]",
    "[*;r17]",
    "[#8][#8]",
    "[#6;+]",
    "[#16][#16]",
    "[#7;!n][S;!$(S(=O)=O)]",
    "[#7;!n][#7;!n]",
    "C#C",
    "C(=[O,S])[O,S]",
    "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
    "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
    "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
    "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
    "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
    "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
]

[[stage.scoring.component]]
[stage.scoring.component.MolecularWeight]

[[stage.scoring.component.MolecularWeight.endpoint]]
name = "Molecular weight"  # user chosen name for output
weight = 0.342  # weight to fine-tune the relevance of this component

# A transform ensures that the output from the scoring component ranges
# from 0 to 1 to serve as a proper score.  Here we use a double sigmoid
# to transform weights into the range 200-500 a.u.
transform.type = "double_sigmoid"
transform.high = 500.0
transform.low = 200.0
transform.coef_div = 500.0
transform.coef_si = 20.0
transform.coef_se = 20.0


### Stage 2
# Alternatively only the first stage above can be run and the new input file
# sets agent_file = 'test1.chkpt'.

[[stage]]

chkpt_file = 'test2.chkpt'

termination = "simple"
max_score = 0.7
min_steps = 10
max_steps = 100

[stage.scoring]  # the scoring components can be read from a score file
type = "geometric_mean"  # aggregation function
filename = "configs/toml/stage2_scoring.toml"  # file with scoring setup for this stage
filetype = "toml"  # file format: TOML or JSON, no default, must be present


### Stage 3
# just as above

"""

In [83]:
sampling_path = os.path.join(project_dir, "sampling_config.toml")

# Parse the TOML string
sampling_dict = toml.loads(sampling_toml)

# Write the TOML content to a file
with open(sampling_path, 'w') as f:
    toml.dump(sampling_dict, f)

In [82]:
sampling_toml = f"""

run_type = "sampling"
device = "cpu"  # set torch device e.g. "cpu"
json_out_config = "_sampling.json"  # write this TOML to JSON

[parameters]

## Reinvent: de novo sampling
model_file = "/Users/devanshjain/REINVENT4/priors/reinvent.prior"

output_file = 'sampling.csv'  # sampled SMILES and NLL in CSV format

num_smiles = 157  # number of SMILES to be sampled, 1 per input SMILES
unique_molecules = true  # if true remove all duplicatesd canonicalize smiles
randomize_smiles = true # if true shuffle atoms in SMILES randomly

"""

In [1]:
!reinvent -l {log_file_reinvent} {sampling_path}

Traceback (most recent call last):
  File "/Users/devanshjain/miniconda3/envs/reinvent4/bin/reinvent", line 8, in <module>
    sys.exit(main_script())
             ~~~~~~~~~~~^^
  File "/Users/devanshjain/miniconda3/envs/reinvent4/lib/python3.13/site-packages/reinvent/Reinvent.py", line 195, in main_script
    main(args)
    ~~~~^^^^^^
  File "/Users/devanshjain/miniconda3/envs/reinvent4/lib/python3.13/site-packages/reinvent/Reinvent.py", line 73, in main
    input_config = config_parse.read_config(args.config_filename, fmt)
  File "/Users/devanshjain/miniconda3/envs/reinvent4/lib/python3.13/site-packages/reinvent/utils/config_parse.py", line 203, in read_config
    with open(filename, "rb") as tf:
         ~~~~^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/Users/devanshjain/laboratoire_d_intelligence_artificielle_en_chimie/{sampling_path}'
